"creativity is the residue of wasted time" - Einstein

In [44]:
# random fact corner

# "people from different cultures can agree that a song is angry, 
# but can differ on whether that feeling is positive or negative"
# - Cowen, https://greatergood.berkeley.edu/article/item/how_many_emotions_can_music_make_you_feel

# valence -> culture specific; positive or negative values

In [ ]:
"""
dependencies
!python -m pip install --upgrade pip
!pip install --upgrade google-cloud
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-storage

!sed 's/ //g' data/mpd.slice.54000-54999.json | tr '\n' ' '> data/mpd.slice.54000-54999_clean.json

!gsutil -m cp data/mpd.slice.54000-54999_clean.json gs://the-bourgeoisie
"""

# source: https://big-data-demystified.ninja/2020/04/12/
# how-to-debug-error-while-reading-data-error-message-
# failed-to-parse-json-unexpected-end-of-string-unexpected-end-of-string-expected-key/

In [ ]:
"""
process overview

using the following code I got some view into dataset, but the hardcoded index
prevented me from seeing some instances of the playlist object had an additional
field, description. being a spotify user I knew this wasn't used a lot and was
optional. I found it by looking into the dataset from the GCS perspective and
using the find/search. meanwhile I'm not sure if I need both sed and ts but
whatever, it works. the idea is to use the newer json format which seems to have
lower memory storage by not using tons of spacing and newline chars, even if it
makes it prettier to read for human.

!head -30 data/mpd.slice.54000-54999.json

print(data.keys())
print(data["info"].keys())
print(data["playlists"][1].keys())
print(data["playlists"][1]["tracks"][1].keys())
"""

In [2]:
import os
import pandas as pd
import json
from google.cloud import bigquery
from google.cloud import storage

In [3]:
# not secure but I'm lazy, steal what you must
home = os.path.expanduser('~')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=f"{home}/.config/gcloud/alpine-beacon-336222-c6a1c087e3ef.json"

In [ ]:
with open("data/mpd.slice.54000-54999.json", "r") as the_jfile:
    data = json.load(the_jfile)

In [6]:
name_top_len = 0
track_name_top_len = 0
artist_top_len = 0
uri_top_len = 0
album_name_top_len = 0

idx = 0

#dict_keys(['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms', 'num_artists'])

while idx < len(data["playlists"]):
    for KEY, VAL in data["playlists"][idx].items():
#        print(f"{KEY} : {VAL}")
        if KEY=="name":
            cur_len = len(VAL)
            if cur_len > name_top_len:
                name_top_len = cur_len
        elif KEY == "tracks":
            tracks = data["playlists"][idx]["tracks"]
            for track in tracks:
                cur_len = len(track["track_name"])                
                if cur_len > track_name_top_len:
                    track_name_top_len = cur_len
                cur_len = len(track["artist_name"])
                if cur_len > artist_top_len:
                    artist_top_len = cur_len
                cur_len = len(track["track_uri"]) #36
                if cur_len > uri_top_len:
                    uri_top_len = cur_len
                cur_len = len(track["artist_uri"]) #37
                if cur_len > uri_top_len:
                    uri_top_len = cur_len
                cur_len = len(track["album_uri"]) #36
                if cur_len > uri_top_len:
                    uri_top_len = cur_len
                cur_len = len(track["album_name"])
                if cur_len > album_name_top_len:
                    album_name_top_len = cur_len
    idx += 1

#print(idx)
        
print("top playlist name len found to be {}".format(name_top_len))
print("top track name len found to be {}".format(track_name_top_len))
print(f"top artist name len found to be {artist_top_len}")
print(f"top uri len found to be {uri_top_len}")
print("top album name len found to be {}".format(album_name_top_len))

top playlist name len found to be 66
top track name len found to be 202
top artist name len found to be 91
top uri len found to be 37
top album name len found to be 151


In [45]:
# Construct a BigQuery client object.
client = bigquery.Client()
project = client.project

print(project)

alpine-beacon-336222


In [53]:
datasets = list(client.list_datasets())  # Make an API request.

dsets=[]

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        ds = dataset.dataset_id
        dsets.append(ds)
        print("\t{}".format(ds))
else:
    print("{} project does not contain any datasets.".format(project))

Datasets in project alpine-beacon-336222:
	music
	r_e_s_p_e_c_t


In [55]:
if "music" not in dsets:
    dataset_id = "{}.music".format(client.project)
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
else:
    print("music is gucci")

music is gucci


In [58]:
def bucket_metadata(bucket_name):
    """Prints out a bucket's metadata."""
    # bucket_name = 'your-bucket-name'

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    print(f"ID: {bucket.id}")
    print(f"Name: {bucket.name}")
    print(f"Storage Class: {bucket.storage_class}")
    print(f"Location: {bucket.location}")
    print(f"Location Type: {bucket.location_type}")
    print(f"Cors: {bucket.cors}")
    print(f"Default Event Based Hold: {bucket.default_event_based_hold}")
    print(f"Default KMS Key Name: {bucket.default_kms_key_name}")
    print(f"Metageneration: {bucket.metageneration}")
    print(
        f"Public Access Prevention: {bucket.iam_configuration.public_access_prevention}"
    )
    print(f"Retention Effective Time: {bucket.retention_policy_effective_time}")
    print(f"Retention Period: {bucket.retention_period}")
    print(f"Retention Policy Locked: {bucket.retention_policy_locked}")
    print(f"Requester Pays: {bucket.requester_pays}")
    print(f"Self Link: {bucket.self_link}")
    print(f"Time Created: {bucket.time_created}")
    print(f"Versioning Enabled: {bucket.versioning_enabled}")
    print(f"Labels: {bucket.labels}")

def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    for blob in blobs:
        print(blob.name)
        

In [63]:
bucket_name="the-bourgeoisie"

print("=====printing random metadata . . . ")
bucket_metadata(bucket_name)

print("\n===printing blobs . . .")
list_blobs(bucket_name)

=====printing random metadata . . . 
ID: the-bourgeoisie
Name: the-bourgeoisie
Storage Class: STANDARD
Location: US
Location Type: multi-region
Cors: []
Default Event Based Hold: False
Default KMS Key Name: None
Metageneration: 1
Public Access Prevention: inherited
Retention Effective Time: None
Retention Period: None
Retention Policy Locked: None
Requester Pays: False
Self Link: https://www.googleapis.com/storage/v1/b/the-bourgeoisie
Time Created: 2022-02-24 20:23:38.300000+00:00
Versioning Enabled: False
Labels: {}

===printing blobs . . .
mpd.slice.54000-54999.json
mpd.slice.54000-54999_giraff.json
mpd.slice.54000-54999_hi.json
mpd.slice.54000-54999_test.json


In [42]:
# Construct a BigQuery client object.
client = bigquery.Client()

# Set table_id to the ID of the table to create.
table_id = "alpine-beacon-336222.music.slices" # "project.dataset.table_name"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("info", "RECORD", mode="REPEATED",
                            fields=(
                                bigquery.SchemaField('generated_on', 'STRING'), # todo
                                bigquery.SchemaField('slice', 'STRING'),
                                bigquery.SchemaField('version', 'STRING')
                            )),
        bigquery.SchemaField("playlists", "RECORD", mode="REPEATED",
                             fields=(
                                bigquery.SchemaField("name", "STRING"),
                                bigquery.SchemaField("collaborative", "BOOLEAN"),
                                bigquery.SchemaField("pid", "INTEGER"),
                                bigquery.SchemaField("modified_at", "INTEGER"),
                                bigquery.SchemaField("num_tracks", "INTEGER"),
                                bigquery.SchemaField("num_albums", "INTEGER"),
                                bigquery.SchemaField("num_followers", "INTEGER"),
                                bigquery.SchemaField("tracks", "RECORD", mode="REPEATED",
                                                    fields=(
                                                        bigquery.SchemaField('pos', 'INTEGER'),
                                                        bigquery.SchemaField('artist_name', 'STRING'),
                                                        bigquery.SchemaField('track_uri', 'STRING'),
                                                        bigquery.SchemaField('artist_uri', 'STRING'),
                                                        bigquery.SchemaField('track_name', 'STRING'),
                                                        bigquery.SchemaField('album_uri', 'STRING'),
                                                        bigquery.SchemaField('duration_ms', 'INTEGER'),
                                                        bigquery.SchemaField('album_name', 'STRING')
                                                    )),
                                bigquery.SchemaField("num_edits", "INTEGER"),
                                bigquery.SchemaField("duration_ms", "INTEGER"),
                                bigquery.SchemaField("num_artists", "INTEGER"),
                                bigquery.SchemaField("description", "STRING") # troublemaker
                             ))
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
)

In [43]:
from botocore.exceptions import ClientError
uri = "gs://the-bourgeoisie/mpd.slice.54000-54999_hi.json"

load_job = client.load_table_from_uri(
    uri, table_id,
    job_config=job_config
)  # Make an API request.

try:
    load_job.result()  # Waits for the job to complete.
except ClientError as eek:
    for eRr in load_job.errors:
    print(f"{eRr} \n")
    raise eek

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1 rows.


TypeError: 'NoneType' object is not iterable